In [1]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = "38002279fa574fbcba0fbc424ae056d5"
os.environ["OPENAI_API_BASE"] = "https://lumin-azure-openai-rnd.openai.azure.com/"

client = OpenAI(
    #api_type="azure",
    api_key=os.environ.get("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_API_BASE"),
    api_version="2023-03-15-preview",
)

TypeError: __init__() got an unexpected keyword argument 'api_type'

In [3]:
!pip freeze

absl-py==1.4.0
accelerate==0.21.0
aiohttp==3.9.3
aiosignal==1.3.1
alembic==1.11.1
annotated-types==0.6.0
anyio==4.3.0
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.2.1
async-generator==1.10
async-timeout==4.0.3
attrs==23.2.0
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==4.1.0
boltons==23.0.0
bqplot==0.12.39
brotlipy==0.7.0
cachetools==5.3.1
certifi==2024.2.2
certipy==0.1.3
cffi==1.15.1
charset-normalizer==3.3.2
cloudpickle==1.6.0
conda==23.5.0
conda-package-handling==2.1.0
conda-package-streaming==0.8.0
configparser==6.0.0
cryptography==39.0.1
datasets==2.14.3
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.7
distro==1.9.0
entrypoints==0.4
evaluate==0.4.0
exceptiongroup==1.2.0
executing==1.2.0
fastjsonschema==2.17.1
filelock==3.9.0
frozenlist==1.4.1
fsspec==2023.6.0
google-auth==2.22.0
google-auth-oauthlib==1.0.0
greenlet==2.0.2
grpcio==1.56.2
h11==0.14.0
httpcore==1.0.4
httpx==0.27.0
huggingface-hub==0.16.4
idna==3.6
importlib-metadata==6.7.0
importlib-reso

In [3]:
completion = openai.ChatCompletion.create(
    engine="lumin-rnd-gpt-4-32k",
    temperature=0.1,
    messages=[{"role": "user", "content": "tell about yourself"}],
    logprobs=True
)
completion

completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

InvalidRequestError: This model does not support the 'logprobs' parameter.

In [2]:
!pip install openai==1.2.0 --upgrade

  Using cached https://files.pythonhosted.org/packages/42/9d/b227c396c6eafbece9a8fe877379582481b108c0c0cf4ff8770ce63926ad/openai-1.2.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/7b/ddacf6dcebb42466abd03f368782142baa82e08fc0c1f8eaa05b4bae87d5/httpx-0.27.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e5/f3/8296f550276194a58